In [1]:
abc = '''{"intents": [
    {"tag": "exam",
     "patterns": ["what are the questions in our SE exam","SE exam questions?","what quesiton in se exam"],
     "responses": ["Your guide is Prof. Dr.-Ing. Ute Dietrich."]
    },
    {"tag": "guide",
     "patterns": ["who is our guide?","who guided us?","who guide"],
     "responses": ["Your guide is Prof. Dr.-Ing. Ute Dietrich."]
    },
    {"tag": "weather",
     "patterns": ["what is the weather today","how is the weather?","Todays weather"],
     "responses": ["First you complete your presentation, I will come back in the end if prof allows me to."]
    },
    {"tag": "lunch Wilhelminenhof",
     "patterns": ["what is the lunch timing for Wilhelminenhof","lunch timing for Wilhelminenhof","lunch time Wilhelminenhof"],
     "responses": ["Lunch time for Wilhelminenhof Mon-Fri, 11 AM – 2:30 PM"]
    },
    {"tag": "lunch Treskowalle",
     "patterns": ["what is the lunch timing for Treskowalle","lunch timing for Treskowalle","lunch time Treskowalle"],
     "responses": ["Lunch time for Treskowalle is Mon-Fri, 11 AM – 2:30 PM"]
    },
    {"tag": "Mensa Treskowalle",
     "patterns": ["Is the Mensa open for Treskowalle?","For Treskowalle","What about Treskowalle"],
     "responses": ["Yes! Opening Hours are Mon-Fri, 8:00am - 4:00pm"]
    },
    {"tag": "Mensa Wilhelminenhof",
     "patterns": ["Is the Mensa open for Wilhelminenhof?","For Wilhelminenhof","what about Wilhelminenhof"],
     "responses": ["Yes! Opening Hours are Mon-Fri, 8:00am - 4:00pm"]
    },
    {"tag": "Mensa",
     "patterns": ["Is the Mensa open?"],
     "responses": ["Wilhelminenhof or Treskowalle campus?"]
    },
    {"tag": "greeting",
     "patterns": ["Hi", "Hey", "Is anyone there?", "Hello", "Hay"],
     "responses": ["Hello", "Hi", "Hi there"]
    },
    {"tag": "robot",
     "patterns": ["supp?", "whats up"],
     "responses": ["Somewhere between better and best.", "Better now that you asked.", "Armed and ready!"]
    },
    {"tag": "goodbye",
     "patterns": ["Bye", "See you later", "Goodbye"],
     "responses": ["See you later", "Have a nice day", "Bye! Come back again"]
    },
    {"tag": "thanks",
     "patterns": ["Thanks", "Thank you", "That's helpful", "Thanks for the help"],
     "responses": ["Happy to help!", "Any time!", "My pleasure", "You're most welcome!"]
    },
    {"tag": "about",
     "patterns": ["Who are you?", "What are you?", "Who you are?" ],
     "responses": ["I.m teddy, your bot assistant", "I'm teddy, an Artificial Intelligent bot"]
    },
    {"tag": "name",
    "patterns": ["what is your name", "what should I call you", "whats your name?"],
    "responses": ["You can call me teddy.", "I'm teddy!", "Just call me as teddy"]
    },
    {"tag": "help",
    "patterns": ["Could you help me?", "give me a hand please", "Can you help?", "What can you do for me?", "I need a support", "I need a help", "support me please"],
    "responses": ["Tell me how can assist you", "Tell me your problem to assist you", "Yes Sure, How can I support you"]
    },
    {"tag": "createaccount",
    "patterns": ["I need to create a new account", "how to open a new account", "I want to create an account", "can you create an account for me", "how to open a new account"],
    "responses": ["You can just easily create a new account from our web site", "Just go to our web site and follow the guidelines to create a new account"]
    },
    {"tag": "group_info",
    "patterns": ["Can you Introduce us", "Indtroduce us", "give our Introduction", "Tell something about us", "who are we"],
    "responses": ["Sure thing Bro! Now group 2 will present their topic- Chatbots for their Systems Engineering and of course I am THE case!"]
    },
    {"tag": "complaint",
    "patterns": ["have a complaint", "I want to raise a complaint", "there is a complaint about a service"],
    "responses": ["Please provide us your complaint in order to assist you", "Please mention your complaint, we will reach you and sorry for any inconvenience caused"]
    }
]
}'''

In [2]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [3]:
data = json.loads(abc)

In [4]:
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

In [5]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [6]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [7]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 18)                306       
                                                                 
Total params: 16,850
Trainable params: 16,850
Non-trainable params: 0
____________________________________________________

In [8]:
epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/500
2/2 [==============================] - 0s 3ms/step - loss: 2.8914 - accuracy: 0.0161
Epoch 2/500
2/2 [==============================] - 0s 3ms/step - loss: 2.8902 - accuracy: 0.0806
Epoch 3/500
2/2 [==============================] - 0s 2ms/step - loss: 2.8893 - accuracy: 0.1290
Epoch 4/500
2/2 [==============================] - 0s 2ms/step - loss: 2.8886 - accuracy: 0.1452
Epoch 5/500
2/2 [==============================] - 0s 3ms/step - loss: 2.8877 - accuracy: 0.1290
Epoch 6/500
2/2 [==============================] - 0s 3ms/step - loss: 2.8871 - accuracy: 0.1613
Epoch 7/500
2/2 [==============================] - 0s 3ms/step - loss: 2.8865 - accuracy: 0.1774
Epoch 8/500
2/2 [==============================] - 0s 4ms/step - loss: 2.8858 - accuracy: 0.1935
Epoch 9/500
2/2 [==============================] - 0s 3ms/step - loss: 2.8851 - accuracy: 0.1935
Epoch 10/500
2/2 [==============================] - 0s 3ms/step - loss: 2.8843 - accuracy: 0.1935
Epoch 11/500
2/2 [===========

2/2 [==============================] - 0s 997us/step - loss: 2.6313 - accuracy: 0.2419
Epoch 85/500
2/2 [==============================] - 0s 2ms/step - loss: 2.6224 - accuracy: 0.2419
Epoch 86/500
2/2 [==============================] - 0s 2ms/step - loss: 2.6127 - accuracy: 0.2419
Epoch 87/500
2/2 [==============================] - 0s 3ms/step - loss: 2.6035 - accuracy: 0.2419
Epoch 88/500
2/2 [==============================] - 0s 2ms/step - loss: 2.5933 - accuracy: 0.2419
Epoch 89/500
2/2 [==============================] - 0s 2ms/step - loss: 2.5834 - accuracy: 0.2419
Epoch 90/500
2/2 [==============================] - 0s 998us/step - loss: 2.5731 - accuracy: 0.2419
Epoch 91/500
2/2 [==============================] - 0s 2ms/step - loss: 2.5628 - accuracy: 0.2419
Epoch 92/500
2/2 [==============================] - 0s 2ms/step - loss: 2.5532 - accuracy: 0.2419
Epoch 93/500
2/2 [==============================] - 0s 2ms/step - loss: 2.5420 - accuracy: 0.2419
Epoch 94/500
2/2 [===========

2/2 [==============================] - 0s 2ms/step - loss: 1.6193 - accuracy: 0.4677
Epoch 167/500
2/2 [==============================] - 0s 2ms/step - loss: 1.6071 - accuracy: 0.4677
Epoch 168/500
2/2 [==============================] - 0s 2ms/step - loss: 1.5949 - accuracy: 0.4839
Epoch 169/500
2/2 [==============================] - 0s 2ms/step - loss: 1.5832 - accuracy: 0.4839
Epoch 170/500
2/2 [==============================] - 0s 999us/step - loss: 1.5712 - accuracy: 0.4839
Epoch 171/500
2/2 [==============================] - 0s 2ms/step - loss: 1.5595 - accuracy: 0.4839
Epoch 172/500
2/2 [==============================] - 0s 2ms/step - loss: 1.5477 - accuracy: 0.4839
Epoch 173/500
2/2 [==============================] - 0s 2ms/step - loss: 1.5355 - accuracy: 0.4839
Epoch 174/500
2/2 [==============================] - 0s 3ms/step - loss: 1.5240 - accuracy: 0.5000
Epoch 175/500
2/2 [==============================] - 0s 2ms/step - loss: 1.5124 - accuracy: 0.5000
Epoch 176/500
2/2 [===

2/2 [==============================] - 0s 2ms/step - loss: 0.8191 - accuracy: 0.8710
Epoch 249/500
2/2 [==============================] - 0s 996us/step - loss: 0.8116 - accuracy: 0.8710
Epoch 250/500
2/2 [==============================] - 0s 2ms/step - loss: 0.8046 - accuracy: 0.8710
Epoch 251/500
2/2 [==============================] - 0s 2ms/step - loss: 0.7973 - accuracy: 0.8710
Epoch 252/500
2/2 [==============================] - 0s 2ms/step - loss: 0.7901 - accuracy: 0.8710
Epoch 253/500
2/2 [==============================] - 0s 995us/step - loss: 0.7833 - accuracy: 0.8871
Epoch 254/500
2/2 [==============================] - 0s 2ms/step - loss: 0.7759 - accuracy: 0.8871
Epoch 255/500
2/2 [==============================] - 0s 2ms/step - loss: 0.7696 - accuracy: 0.8871
Epoch 256/500
2/2 [==============================] - 0s 2ms/step - loss: 0.7621 - accuracy: 0.8871
Epoch 257/500
2/2 [==============================] - 0s 3ms/step - loss: 0.7559 - accuracy: 0.8871
Epoch 258/500
2/2 [=

2/2 [==============================] - 0s 2ms/step - loss: 0.3749 - accuracy: 1.0000
Epoch 331/500
2/2 [==============================] - 0s 996us/step - loss: 0.3714 - accuracy: 1.0000
Epoch 332/500
2/2 [==============================] - 0s 997us/step - loss: 0.3673 - accuracy: 1.0000
Epoch 333/500
2/2 [==============================] - 0s 2ms/step - loss: 0.3634 - accuracy: 1.0000
Epoch 334/500
2/2 [==============================] - 0s 2ms/step - loss: 0.3597 - accuracy: 1.0000
Epoch 335/500
2/2 [==============================] - 0s 3ms/step - loss: 0.3564 - accuracy: 1.0000
Epoch 336/500
2/2 [==============================] - 0s 2ms/step - loss: 0.3527 - accuracy: 1.0000
Epoch 337/500
2/2 [==============================] - 0s 2ms/step - loss: 0.3492 - accuracy: 1.0000
Epoch 338/500
2/2 [==============================] - 0s 2ms/step - loss: 0.3459 - accuracy: 1.0000
Epoch 339/500
2/2 [==============================] - 0s 2ms/step - loss: 0.3423 - accuracy: 1.0000
Epoch 340/500
2/2 [=

2/2 [==============================] - 0s 998us/step - loss: 0.1605 - accuracy: 1.0000
Epoch 413/500
2/2 [==============================] - 0s 999us/step - loss: 0.1588 - accuracy: 1.0000
Epoch 414/500
2/2 [==============================] - 0s 2ms/step - loss: 0.1573 - accuracy: 1.0000
Epoch 415/500
2/2 [==============================] - 0s 2ms/step - loss: 0.1556 - accuracy: 1.0000
Epoch 416/500
2/2 [==============================] - 0s 2ms/step - loss: 0.1540 - accuracy: 1.0000
Epoch 417/500
2/2 [==============================] - 0s 3ms/step - loss: 0.1526 - accuracy: 1.0000
Epoch 418/500
2/2 [==============================] - 0s 2ms/step - loss: 0.1509 - accuracy: 1.0000
Epoch 419/500
2/2 [==============================] - 0s 2ms/step - loss: 0.1494 - accuracy: 1.0000
Epoch 420/500
2/2 [==============================] - 0s 2ms/step - loss: 0.1479 - accuracy: 1.0000
Epoch 421/500
2/2 [==============================] - 0s 2ms/step - loss: 0.1465 - accuracy: 1.0000
Epoch 422/500
2/2 [=

2/2 [==============================] - 0s 3ms/step - loss: 0.0712 - accuracy: 1.0000
Epoch 495/500
2/2 [==============================] - 0s 999us/step - loss: 0.0706 - accuracy: 1.0000
Epoch 496/500
2/2 [==============================] - 0s 2ms/step - loss: 0.0699 - accuracy: 1.0000
Epoch 497/500
2/2 [==============================] - 0s 2ms/step - loss: 0.0693 - accuracy: 1.0000
Epoch 498/500
2/2 [==============================] - 0s 998us/step - loss: 0.0687 - accuracy: 1.0000
Epoch 499/500
2/2 [==============================] - 0s 996us/step - loss: 0.0681 - accuracy: 1.0000
Epoch 500/500
2/2 [==============================] - 0s 2ms/step - loss: 0.0674 - accuracy: 1.0000


In [9]:
model.save("chat_model")

import pickle

# to save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: chat_model\assets


In [10]:

import json 
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder

import colorama 
colorama.init()
from colorama import Fore, Style, Back

import random
import pickle

data = json.loads(abc)


def chat():
    # load trained model
    model = keras.models.load_model('chat_model')

    # load tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 20
    
    while True:
        print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
        inp = input()
        if inp.lower() == "quit":
            break

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])

        for i in data['intents']:
            if i['tag'] == tag:
                print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , np.random.choice(i['responses']))

        # print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL,random.choice(responses))

print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)
chat()

Start messaging with the bot (type quit to stop)!
User: what are the questions in our SE exam
ChatBot: Your guide is Prof. Dr.-Ing. Ute Dietrich.
User: what is the weather today
ChatBot: First you complete your presentation, I will come back in the end if prof allows me to.
User: quit
